# Conception de maille avec Xsuite

Recréons la maille `PIMMS` développée par la collaboration TERA pour la thérapie par protons et ions (voir : [CERN/PS 99-010](https://cds.cern.ch/record/385378/)) et sur laquelle sont basés les synchrotrons médicaux `CNAO` et `MEDAUSTRON`.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xtrack as xt

%config InlineBackend.figure_format = "retina"

### Création d'Environement and définition de la particule de référence

In [ ]:
env = xt.Environment()
env.particle_ref = xt.Particles(kinetic_energy0=200e6)  # proton, 200MeV
env.vars.default_to_zero = True  # Variables non définies sont à zéro

### Définition des élements

In [ ]:
# Géometries des éléments
n_bends = 16
env["ang_mb"] = 2 * np.pi / n_bends
env["l_mb"] = 1.65
env["l_mq"] = 0.35

# Créations de dipoles et quadripôles génériques
env.new("mb", parent=xt.RBend, length="l_mb", angle="ang_mb", k0_from_h=True)
env.new("mq", parent=xt.Quadrupole, length="l_mq");

In [ ]:
# Différentes familles de quadripôles
env.new("qfa", parent="mq", k1="kqfa")
env.new("qfb", parent="mq", k1="kqfb")
env.new("qd", parent="mq", k1="kqd");

### Conception d'une cellule

In [ ]:
# Cellule FODO (focalisant-défocalisant-focalisant)
cell_a = env.new_line(
    length=7.405,
    components=[
        env.place("qfa", at=0.3875),
        env.place("mb", at=1.8125),
        env.place("qd", at=3.2925),
        env.place("mb", at=5.0475),
        env.place("qfa", at=6.3275),
    ],
)
cell_a.survey().plot()

In [ ]:
# Cellule similaire avec la seconde famille de quadripôles focalisants
cell_b = env.new_line(
    name="cell_b",
    length=8.405,
    components=[
        env.place("qfb", at=1.2725),
        env.place("mb", at=2.7275),
        env.place("qd", at=4.8575),
        env.place("mb", at=6.5125),
        env.place("qfb", at=7.7925),
    ],
)

### Conception d'un arc

In [ ]:
# Concaténation des deux cellules en un arc
arc = cell_a + cell_b
arc.survey().plot()

### Conception de sections droites

In [ ]:
long_straight = env.new_line(length=2.0, components=[env.new("mid.lss", parent=xt.Marker, at=1.0)])
short_straight = env.new_line(length=1.0, components=[env.new("mid.sss", parent=xt.Marker, at=0.5)])

### Assemblage de l'anneau

In [ ]:
half_ring = long_straight + arc + short_straight - arc  # pour une maille symmétrique
ring = 2 * half_ring
ring.survey().plot()

### Replacement des éléments répétés

In [ ]:
# Déduplication en donnant des noms uniques
ring.replace_all_repeated_elements()

### Inspection de la table

In [ ]:
table = ring.get_table()
table.cols["element_type", "s_start", "s_center", "s_end"]

In [ ]:
# Séléction des quadripôles uniquement
table_quads = table.rows[table.element_type == "Quadrupole"]
table_quads.cols["s_start", "s_center", "s_end"]

In [ ]:
# Mise en exergue de leurs positions dans la maille
sv = ring.survey()
sv.plot(labels=table_quads.name);

### Definition et installation des sextupôles

In [ ]:
# Aimant sextupôles générique
env.new("ms", parent=xt.Sextupole, length=0.2)

# Les sextupôles eux-mêmes
env.new("msf.1", parent="ms", k2="ksf")  # focalisant
env.new("msf.2", parent="ms", k2="ksf")  # focalisant
env.new("msd.1", parent="ms", k2="ksd")  # défocalisant
env.new("msd.2", parent="ms", k2="ksd")  # défocalisant
env.new("mse", parent="ms", k2="kse")
# pour l'extraction

In [ ]:
# Insertion dans l'anneau, en positions relatives
ring.insert(
    [
        env.place("msf.1", at=-0.2, from_="qfb.0@start"),
        env.place("msf.2", at=-0.2, from_="qfb.4@start"),
        env.place("msd.1", at=0.3, from_="qd.2@end"),
        env.place("msd.2", at=0.3, from_="qd.6@end"),
        env.place("mse", at=-0.3, from_="qfa.4@start"),
    ]
)

In [ ]:
# Mise en exergue dans la maille
survey = ring.survey()
survey.plot(labels=["msf.1", "msf.2", "msd.1", "msd.2", "mse"])

### Inspection de la structure des circuits

In [ ]:
# Éléments contrôlés par une variable
env.info("ksf")

### Installation d'une cavité RF

In [ ]:
env.new("rf1", parent=xt.Cavity, voltage="vrf", frequency="frf")
ring.insert("rf1", at=0.5, from_="qfa.3@start")
survey = ring.survey()
survey.plot(labels=["rf1"])

### Export de la maille en fichier JSON

In [ ]:
env["ring"] = ring
env.to_json("pimms.json")

---